In [39]:
import pandas as pd

In [40]:
# --------------- #
#  Load Raw Data  #
# --------------- #

customers = pd.read_csv("../data/raw/customer.csv")
online_txn = pd.read_csv("../data/raw/online_transactions.csv")
offline_txn = pd.read_csv("../data/raw/offline_transactions.csv")
loyalty = pd.read_csv("../data/raw/loyalty_points.csv")

In [41]:
# Combine all transactions

transactions = pd.concat([
    online_txn.assign(channel='Online'),
    offline_txn.assign(channel='Offline')  
], ignore_index=True)

print(f'Transactions data shape = {transactions.shape}')
pd.concat([transactions.head(5), transactions.tail(5)])

Transactions data shape = (6000, 7)


,transaction_id,customer_id,transaction_date,amount,platform,channel,store_location
0,eb68282d-c40d-4dd6-8c82-a04561fdeb59,61df8168-e62c-421a-9970-c530eff973c1,2020-07-18,1210.06,Website,Online,NaN
1,07738d65-e9c8-4768-a184-86f7db3c57b0,60809de2-52cb-4080-9dc1-356ffe51374b,2023-03-16,478.11,Website,Online,NaN
2,41134906-8039-4166-92b6-4ffed3649dc5,00bf6959-c043-41ea-a1a1-25f1db65b0f3,2021-09-26,3877.29,Website,Online,NaN
3,73e77662-9974-4408-953f-a0fe45d97ad3,50d68fbc-ec07-412d-8d9e-168beb3d440e,2022-06-30,1750.15,Website,Online,NaN
4,93dafe6b-8904-4fd2-988d-9c97dcc04153,9101a3a1-2731-4032-81d6-ae569ed66a22,2022-05-06,1945.93,Website,Online,NaN
5995,e61de7a2-68ba-472c-8824-fe05f24393cd,1e56ee9e-13c1-435b-92e7-bdc3386bcee7,2020-11-27,2836.09,NaN,Offline,Phuket
5996,e52dd341-20ee-499d-a069-5e4a826d59be,ebc8acf5-2587-4867-8728-95e214e0fd8b,2024-02-20,2671.60,NaN,Offline,Pattaya
5997,bd55d508-bf5d-4c7c-9071-1489c367ea7c,1271e0e5-4e18-4d55-bc65-318c4d92aea7,2020-11-13,142.84,NaN,Offline,Pattaya
5998,02d6a20a-9b85-46a8-8fee-4c00309fdc58,7606a0ef-e0bf-4f34-939f-d6719451d1cd,2020-06-10,835.63,NaN,Offline,Pattaya
5999,6866443b-63a5-4ef6-9c58-7b5d0eff4712,9c44551c-b10a-41b1-aa62-a5dade7811db,2022-03-09,731.16,NaN,Offline,Khon Kaen


In [42]:
# ------------------------- #
# Calculate Transaction KPI #
# ------------------------- #

spend_summary = transactions.groupby("customer_id").agg(
    total_transactions=("transaction_id", "count"),
    total_spend=("amount", "sum"),
    avg_transaction_value=("amount", "mean"),
    last_transaction_date=("transaction_date", "max")
).reset_index()

print(f'spend_summary data shape = {spend_summary.shape}')
spend_summary.head(10)

spend_summary data shape = (500, 5)


,customer_id,total_transactions,total_spend,avg_transaction_value,last_transaction_date
0,00bf6959-c043-41ea-a1a1-25f1db65b0f3,15,28264.89,1884.326000,2023-06-01
1,01c333d3-c6ba-493f-ad20-7164856c1a78,15,26200.62,1746.708000,2024-04-14
2,0254056d-cc0b-4fed-98f3-8a7934723863,15,38397.99,2559.866000,2024-04-01
3,02da8432-e11c-443a-8c66-ff7b41a3c475,17,37158.56,2185.797647,2024-04-23
4,03119e07-57e9-450d-9198-0b2c8b3d4dc9,15,32237.93,2149.195333,2024-03-03
5,03bfaf37-e0d0-4ac0-ae77-fa671baef681,6,10403.05,1733.841667,2022-11-18
6,04e55975-952e-4af4-a0d2-4350343f7241,12,33336.00,2778.000000,2024-04-05
7,0528412a-9c96-41af-99fd-be94646946a7,11,26323.03,2393.002727,2023-12-30
8,05ba42d9-84da-4e4e-91ff-e66c6b909ab2,17,45435.00,2672.647059,2024-02-11
9,05bad2a4-2cb2-4b2d-899a-7825f958663b,9,35471.03,3941.225556,2024-04-21


In [43]:
# ---------------------- #
# Loyalty points summary #
# ---------------------- #

loyalty_summary = loyalty.groupby("customer_id").agg(
    total_points_earned=("points_earned", "sum"),
    total_points_redeemed=("points_redeemed", "sum")
).reset_index()

loyalty_summary["current_balance"] = (
    loyalty_summary["total_points_earned"] - loyalty_summary["total_points_redeemed"]
)

print(f'loyalty_summary data shape = {loyalty_summary.shape}')
loyalty_summary.head(10)

loyalty_summary data shape = (500, 4)


,customer_id,total_points_earned,total_points_redeemed,current_balance
0,00bf6959-c043-41ea-a1a1-25f1db65b0f3,1425,615,810
1,01c333d3-c6ba-493f-ad20-7164856c1a78,3582,1480,2102
2,0254056d-cc0b-4fed-98f3-8a7934723863,2354,670,1684
3,02da8432-e11c-443a-8c66-ff7b41a3c475,2831,638,2193
4,03119e07-57e9-450d-9198-0b2c8b3d4dc9,2186,300,1886
5,03bfaf37-e0d0-4ac0-ae77-fa671baef681,1754,700,1054
6,04e55975-952e-4af4-a0d2-4350343f7241,3718,345,3373
7,0528412a-9c96-41af-99fd-be94646946a7,2394,1265,1129
8,05ba42d9-84da-4e4e-91ff-e66c6b909ab2,2252,704,1548
9,05bad2a4-2cb2-4b2d-899a-7825f958663b,4234,796,3438


In [44]:
# -------------------------- #
# Merge All into Master View #
# -------------------------- #

#   SELECT *
#   FROM customers
#   LEFT JOIN spend_summary ON customers.customer_id = spend_summary.customer_id
#   LEFT JOIN loyalty_summary ON customers.customer_id = loyalty_summary.customer_id

master_df = customers.merge(spend_summary, on="customer_id", how="left")
master_df = master_df.merge(loyalty_summary, on="customer_id", how="left")

master_df.head(5)

,customer_id,name,gender,age,registration_channel,registration_date,total_transactions,total_spend,avg_transaction_value,last_transaction_date,total_points_earned,total_points_redeemed,current_balance
0,1a4a26c3-2207-4396-b175-be712100e243,Kevin Thompson,Male,19,Offline,2022-07-01,10,22390.49,2239.049000,2024-03-19,1853,0,1853
1,68fcf654-16a0-4e63-baef-ac161ba0aad5,Charles Mckee,Male,32,Online,2021-03-23,10,27132.31,2713.231000,2024-01-23,2743,618,2125
2,7231d7ba-0c7e-40a6-9cca-5c3321552882,Kevin Wade,Male,61,Online,2022-01-31,13,33696.06,2592.004615,2023-07-15,1982,517,1465
3,527395b0-d07f-4afa-aecd-3876c0939ff9,Ronald Phillips,Female,20,Online,2022-01-15,9,29789.53,3309.947778,2023-04-12,3989,1423,2566
4,470d3c0f-4102-4cbd-bb46-69e79832d3dc,Dr. Michelle Reyes,Male,31,Online,2023-12-06,10,28746.57,2874.657000,2024-04-24,2943,650,2293


In [45]:
# --------------------------------------- #
# Add Customer Status (Active / Inactive) #
# --------------------------------------- #

from datetime import datetime

deadline_days = 180

master_df["last_transaction_date"] = pd.to_datetime(master_df["last_transaction_date"], errors="coerce")
cutoff = pd.to_datetime("today") - pd.Timedelta(days=deadline_days)

master_df["status"] = master_df["last_transaction_date"].apply(
    lambda x: "Inactive" if pd.isna(x) or x < cutoff else "Active"
)

master_df.head(5)

,customer_id,name,gender,age,registration_channel,registration_date,total_transactions,total_spend,avg_transaction_value,last_transaction_date,total_points_earned,total_points_redeemed,current_balance,status
0,1a4a26c3-2207-4396-b175-be712100e243,Kevin Thompson,Male,19,Offline,2022-07-01,10,22390.49,2239.049000,2024-03-19,1853,0,1853,Inactive
1,68fcf654-16a0-4e63-baef-ac161ba0aad5,Charles Mckee,Male,32,Online,2021-03-23,10,27132.31,2713.231000,2024-01-23,2743,618,2125,Inactive
2,7231d7ba-0c7e-40a6-9cca-5c3321552882,Kevin Wade,Male,61,Online,2022-01-31,13,33696.06,2592.004615,2023-07-15,1982,517,1465,Inactive
3,527395b0-d07f-4afa-aecd-3876c0939ff9,Ronald Phillips,Female,20,Online,2022-01-15,9,29789.53,3309.947778,2023-04-12,3989,1423,2566,Inactive
4,470d3c0f-4102-4cbd-bb46-69e79832d3dc,Dr. Michelle Reyes,Male,31,Online,2023-12-06,10,28746.57,2874.657000,2024-04-24,2943,650,2293,Inactive


In [46]:
# Save

master_df.to_csv("../data/processed/customer_master_view.csv", index=False)
print("✅ Customer Master View created and saved.")

✅ Customer Master View created and saved.
